# Recipe Recommender System

## Agenda

1. Datasets
2. Tech Stack
    - Cleaning and preparing the datasets
    - Any fun EDAs 
3. Overview of Recommender Systems
    - Collaborative-based Filtering
    - Content-based Filtering
    - What did not work aka Lessons learnt the hard way
4. Cross Polination
5. Summary

# 1. Datasets

1. Instacart's users and the products they have bought (in the year 2017).
2. Epicurous's recipe dataset (from 2017). 

# 2. Tech Stack

List of tools used and their packages:
1. Relational Database, SQL (SQLite)
2. Graph Database, Neo4j (py2neo)
3. iGraph, for clustering (python-igraph)

## Cleaning datasets for the database

### SQL

- Set up SQL.
- Load the CSVs into pandas and have a quick look.

### Instacart

In [ ]:
connection = sqlite3.connect('./datasets/instacart/sql/instacart.db.sqlite')

In [ ]:
# Load the orders dataset
orders = '../data/raw/orders.csv'
df_order = pd.read_csv(orders, encoding = 'utf8')

# Save it to SQL
df_order.to_sql(name='orders',con=connection,if_exists='replace',index=False)

In [ ]:
# Load the products dataset
products = '../data/raw/products.csv'
df_product = pd.read_csv(products, encoding = 'utf8')

# Save it to SQL
df_product.to_sql(name='product',con=connection,if_exists='replace',index=False)

In [ ]:
# Load the order_products dataset
order_products = '../data/raw/order_products.csv'
df_main = pd.read_csv(order_products, encoding = 'utf8')

# Saving to a SQL file
df_main.to_sql(name='main',con=connection,if_exists='replace',index=False)

### Preparing the files For Neo4j

- Neo4j reads CSV files.
- Doesn't like duplicates. 
- Is quite particular about encoding.
- Doesn't like strings such as: '9" Cast Iron Pot'

Condense 3 CSV files into 2:
1. `products_clean.csv`
2. `users_order.csv`

In [ ]:
# Create products_clean.csv

sql_query = '''
SELECT products.product_id, products.product_name, products.aisle_id, products.department_id, aisle.aisle, department.department
FROM products
INNER JOIN aisle ON products.aisle_id = aisle.aisle_id
INNER JOIN department ON products.department_id = department.department_id
'''

In [ ]:
df = pd.read_sql(sql_query, con=connection)
df.head()

In [ ]:
df['department'].value_counts()

In [ ]:
to_delete = ['personal care','household','missing','babies','pets']

df_clean = df[~df['department'].isin(to_delete)].copy()
df_clean.head()

In [ ]:
# Remove those with " in the names
df_clean = df_clean[~df_clean['product_name'].str.contains('"')]

In [ ]:
# Remove any weird encodings
df_clean['product_name'] = df_clean['product_name'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii').strip())

In [ ]:
# Save to csv
df_clean.to_csv('../data/processed/products_clean.csv', index=False)

In [ ]:
# Create the users AND orders csv

sql_query = '''
SELECT orders.user_id, main.product_id, orders.order_hour_of_day, COUNT(orders.order_id) AS total_orders
FROM main
INNER JOIN product ON product.product_id = main.product_id
INNER JOIN orders ON orders.order_id = main.order_id
GROUP BY orders.order_id
'''

In [ ]:
df = pd.read_sql(sql_query, con=connection)

In [ ]:
df.sort_values(by='user_id', inplace=True)
df.head(20)

In [ ]:
df.to_csv('../data/processed/users_orders.csv', index=False)

### Epicurious

Not CSV, but a JSON file!

In [ ]:
epi = pd.read_json('../data/raw/full_format_recipes.json', orient='values', encoding='utf-8')
epi.head()

In [ ]:
epi.drop(columns=['fat','calories','date','protein','sodium','desc'], inplace=True)

In [ ]:
epi['categories'][0]

In [ ]:
epi['directions'][0]

In [ ]:
epi['ingredients'][0]

In [ ]:
epi['title'][0]

In [ ]:
epi['title'] = test['title'].str.strip()

In [ ]:
epi.shape

In [ ]:
epi.drop_duplicates(subset='title', keep='first', inplace=True)
epi.shape

In [ ]:
epi.to_json('../data/processed/epi_recipe_json_cleaned.json', force_ascii=False, orient='records')

#### Any fun EDAs

In [ ]:
df.head()

In [ ]:
df['total_orders'].describe()

In [ ]:
df.boxplot(column='total_orders',figsize=(10,10))
plt.show()

# Data Stucture

![](../reports/assets/data_structure.png)

# 3. Overview of Recommender Systems

There are three approaches to recommender systems: 
1. Collaborative-based Filtering
2. Content-based Filtering
3. A Hybrid of Both

Similarity metrics:
1. Cosine similarity scores
2. Pearson similarity scores
3. Jaccard similarity scores

## Collaborative-based Filtering

What else do other users buy which is _similar_ to what you have bought?

In [ ]:
# just an example, replace with credentials for your own Neo4j instance
graph = Graph(bolt=True, host="localhost", http_port=7687, user='neo4j', password='kiss')

In [ ]:
results = graph.run("""
MATCH (user:User {id: {user_id}})-[b1:BOUGHT]->(:Product)<-[b2:BOUGHT]-(otheruser:User)
MATCH (otheruser)-[:BOUGHT]->(rec:Product)
WHERE NOT EXISTS( (user)-[:BOUGHT]->(rec)) AND b2.order_total >= b1.order_total
RETURN rec.name, COUNT(*) AS usersWhoAlsoBought
ORDER BY usersWhoAlsoBought DESC 
LIMIT 10""", user_id=random_user)

for row in results:
    print(row)

### Cosine similarity

In [ ]:
results = graph.run("""
MATCH (p1:User {id: {user_id}})-[x:BOUGHT]->(p:Product)<-[y:BOUGHT]-(p2:User)
WITH COUNT(p) AS numberproducts, SUM(x.order_total * y.order_total) AS xyDotProduct,
SQRT(REDUCE(xDot = 0.0, a IN COLLECT(x.order_total) | xDot + a^2)) AS xLength,
SQRT(REDUCE(yDot = 0.0, b IN COLLECT(y.order_total) | yDot + b^2)) AS yLength,
p1, p2 WHERE numberproducts > 10
RETURN p2.id, xyDotProduct / (xLength * yLength) AS cosim
ORDER BY cosim DESC 
LIMIT 10""", user_id=random_user)

for row in results:
    print(row)

### Pearson similarity

This is particularly well-suited for product recommendations because it takes into account the fact that different users will have different mean total orders: on average some people do buy only from Instacart, while some prefer to go out of their house. Since Pearson similarity considers differences about the mean, this metric will account for these discrepancies.

In [ ]:
results = graph.run("""
MATCH (u1:User {id: {user_id}})-[r:BOUGHT]->(m:Product)
WITH u1, avg(r.order_total) AS u1_mean

MATCH (u1)-[r1:BOUGHT]->(m:Product)<-[r2:BOUGHT]-(u2)
WITH u1, u1_mean, u2, COLLECT({r1: r1, r2: r2}) AS totalorders WHERE size(totalorders) > 10

MATCH (u2)-[r:BOUGHT]->(m:Product)
WITH u1, u1_mean, u2, avg(r.order_total) AS u2_mean, totalorders

UNWIND totalorders AS r

WITH sum( (r.r1.order_total - u1_mean) * (r.r2.order_total - u2_mean) ) AS nom,
     sqrt( sum( (r.r1.order_total - u1_mean)^2) * sum( (r.r2.order_total - u2_mean) ^2)) AS denom,
     u1, u2 WHERE denom <> 0

RETURN u2.id, nom/denom AS pearson
ORDER BY pearson DESC 
LIMIT 10""", user_id=random_user)

for row in results:
    print(row)

### Working algorithm for collaborative-based recommendations

Pearson Similiarty and KNN

In [ ]:
results = graph.run("""
MATCH (u1:User {id: {user_id}})-[r:BOUGHT]->(m:Product)
WITH u1, avg(r.order_total) AS u1_mean

MATCH (u1)-[r1:BOUGHT]->(m:Product)<-[r2:BOUGHT]-(u2)
WITH u1, u1_mean, u2, COLLECT({r1: r1, r2: r2}) AS totalorders WHERE size(totalorders) > 10

MATCH (u2)-[r:BOUGHT]->(m:Product)
WITH u1, u1_mean, u2, avg(r.order_total) AS u2_mean, totalorders

UNWIND totalorders AS r

WITH sum( (r.r1.order_total - u1_mean) * (r.r2.order_total - u2_mean) ) AS nom,
     sqrt( sum( (r.r1.order_total - u1_mean)^2) * sum( (r.r2.order_total - u2_mean) ^2)) AS denom,
     u1, u2 WHERE denom <> 0

WITH u1, u2, nom/denom AS pearson
ORDER BY pearson DESC LIMIT 10

MATCH (u2)-[r:BOUGHT]->(m:Product) WHERE NOT EXISTS( (u1)-[:BOUGHT]->(m) )

RETURN m.name AS recommendation, SUM(pearson * r.order_total) AS score
ORDER BY score DESC 
LIMIT 10""", user_id=random_user)

for row in results:
    print(row)

## Clustering

Deep in the world of Graph Theory, there's `python-igraph` which is a network analysis package. [Github](https://github.com/igraph/python-igraph).

Community detection algoritm, based on work by [Pons and Latapy](https://arxiv.org/abs/physics/0512106), _"Computing communities in large networks using random walks"_ (2005). 

The basic idea of the algorithm is that short random walks tend to stay in the same community. 

It utilizes a graph network to find communities (in this case, patterns of Aisles and Deparments) and 'cluster' them together.

In [ ]:
result = graph.run("""
MATCH (a:Aisle)<-[:FOUND_IN]-()-[:TYPE_OF]->(d:Department)
RETURN a.name AS aisleName, d.name AS departmentName, COUNT(*) AS weight
ORDER BY weight DESC
LIMIT 10""")

for row in result:
    print(row)

In [ ]:
cluster = graph.run("""
MATCH (a:Aisle)<-[:FOUND_IN]-()-[:TYPE_OF]->(d:Department)
RETURN a.name AS aisleName, d.name AS departmentName, COUNT(*) AS weight""")

In [ ]:
ig = IGraph.TupleList(cluster, weights=True)
ig

In [ ]:
clusters = IGraph.community_walktrap(ig, weights='weight')
clusters = clusters.as_clustering()
len(clusters)

In [ ]:
# Let's take a look at the 'clusters'
nodes = [node['name'] for node in ig.vs]
nodes = [{'id': x, 'label': x} for x in nodes]
nodes[:5]

for node in nodes:
    idx = ig.vs.find(name=node['id']).index
    node['group'] = clusters.membership[idx]
    
nodes[:20]

### Novelty Recommendation

New but not foreign.

We want to recommend a product that is new but not entirely different from what the person has bought before. 

In [ ]:
result = graph.run("""
MATCH (user:User {id: {user_id}})-[:BOUGHT]->(product)-[:FOUND_IN]->(a:Aisle)-[:IN_CLUSTER]->(cluster)
WITH user, cluster, COUNT(*) AS times
ORDER BY times DESC
LIMIT 1
WITH user, cluster
MATCH (cluster)<-[:IN_CLUSTER]-(a)<-[:FOUND_IN]-()
WITH cluster, a.name AS aisles, COUNT(a) as commonAisles
RETURN aisles, commonAisles
ORDER BY commonAisles DESC
LIMIT 5""", user_id=random_user)

for row in result:
    print(row)

In [ ]:
result = graph.run("""
MATCH (user:User {id: {user_id}})-[:BOUGHT]->(product)-[:FOUND_IN]->(a:Aisle)-[:IN_CLUSTER]->(cluster)
WITH user, cluster, COUNT(*) AS times
ORDER BY times DESC
LIMIT 1
WITH user, cluster
MATCH (cluster)<-[:IN_CLUSTER]-(a)<-[:FOUND_IN]-()
WITH cluster, a.name AS aisles, COUNT(a) as commonAisles
ORDER BY commonAisles ASC
LIMIT 1
WITH aisles AS x
MATCH (Aisle {name: x})<-[:FOUND_IN]-(otherProducts)<-[b:BOUGHT]-()
WHERE b.order_total > 10
RETURN DISTINCT otherProducts.name, MAX(b.order_total) AS orderTotal
ORDER BY orderTotal DESC
LIMIT 10""", user_id=random_user)

for row in result:
    print(row)

## Content-based Filtering

Content-based filtering didn't work for the groceries dataset. This is because content-based filtering relies on a product to be part of multiple categories. For example:

- Movie A -> Crime, Thriller, Adventure, Drama
- Movie B -> Crime, Drama, Romance

But in the case of groceries, a meat is meat, a vegetable is a vegetable. Frozen vegetables were categorized as "Frozen" food not "Vegetable" AND "Frozen".

But for the recipes dataset, it does work.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
pd.DataFrame(graph.data("""
MATCH (m:Recipe)-[:TAGGED_AS]->(g:Category)<-[:TAGGED_AS]-(rec:Recipe)
WHERE m.name = 'Potato and Fennel Soup Hodge'
WITH rec, COLLECT(g.name) AS categories, COUNT(*) AS commonCategories
RETURN rec.name, categories, commonCategories
ORDER BY commonCategories DESC 
LIMIT 10"""))

### Jaccard similarity

The Jaccard index is a number between 0 and 1 that indicates how similar two sets are. The Jaccard index of two identical sets is 1. If two sets do not have a common element, then the Jaccard index is 0. 

We can calculate the Jaccard index for by comparing the number of categories that each recipe overlaps on.

In [ ]:
pd.DataFrame(graph.data("""
MATCH (r:Recipe {name: "Spicy Noodle Soup"})-[:TAGGED_AS]->(c:Category)<-[:TAGGED_AS]-(other:Recipe)
WITH r, other, COUNT(c) AS intersection, COLLECT(c.name) AS i

MATCH (r)-[:TAGGED_AS]->(rc:Category)
WITH r, other, intersection, i, COLLECT(rc.name) AS s1

MATCH (other)-[:TAGGED_AS]->(oc:Category)
WITH r, other, intersection, i,  s1, COLLECT(oc.name) AS s2

WITH r, other, intersection, s1, s2

WITH r, other, intersection, s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2

WHERE other.rating >= r.rating

RETURN other.name AS recommendation, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard 
ORDER BY jaccard DESC 
LIMIT 10"""))

# 4. Cross Polination

Making the leap from Product to Recipes.

In [ ]:
product_name = "Organic Large Extra Fancy Fuji Apple"
words = product_name.split()
words

In [ ]:
pd.DataFrame(graph.data("""
MATCH (m:Recipe)-[:TAGGED_AS]->(c:Category)<-[:TAGGED_AS]-(rec:Recipe)
WHERE c.name IN ['Organic', 'Large', 'Extra', 'Fancy', 'Fuji', 'Apple']
WITH rec, COLLECT(DISTINCT c.name) AS categories, COUNT(DISTINCT c) AS commonCategories
RETURN rec.name, categories, commonCategories
ORDER BY commonCategories DESC 
LIMIT 10"""))

However, this meant that the database queries the Category nodes 6 times. 

Once for 'Organic', for 'Large', for 'Extra'... etc. 

Until it reaches 'Apple' which had the most number of hits.

This is computational expensive. Another way to expand this project, would be to create a **Named Entity Recognition** (NER) model that will parse product names and outputs the relevent keyword that will be the search query for the recipe network.


**INSERT diagram of how the app will function.**

# 5. Summary

What worked and what didn't?
- Collaborative filtering works if you have some form of quantitative ranking, i.e. ratings, total number of orders.
- Content filtering works if you multiple categories for the same item.
- Clustering can help you but like all clustering methods, there's no way to evaluate the outcomes.
- Novelty recommendation is still a field studied (how to do it).

What else after this?
- Named Entity Recognition layer.
- Textblob to create keywords for each recipes, either based on ingredients or description.
- Make the Flask app look less...ugly.
- Try another clustering algorithm, available in the iGraph [package](http://igraph.org/python/doc/igraph.Graph-class.html).